In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import normalize as sklearn_normalize
from sklearn.utils import shuffle
from tqdm.notebook import tqdm
from matplotlib import animation

In [ ]:
DATA_DIR = '../data/training_sites'

In [ ]:
with open(os.path.join(DATA_DIR, "negative_data.pkl"), 'rb') as file:
    x_negative = pickle.load(file)
file.close()
y_negative = np.zeros(len(x_negative))

with open(os.path.join(DATA_DIR, "positive_data.pkl"), 'rb') as file:
    x_positive = pickle.load(file)
file.close()
y_positive = np.ones(len(x_positive))

with open(os.path.join(DATA_DIR, "positive_data_test.pkl"), 'rb') as file:
    x_positive_test = pickle.load(file)
file.close()
y_positive_test = np.ones(len(x_positive_test))

In [ ]:
x = np.concatenate((x_positive, x_negative))
y = np.concatenate((y_positive, y_negative))

In [ ]:
def normalize(x):
    return (np.array(x) / (3000 - 200)

In [ ]:
x, y = shuffle(x, y, random_state=42)
x = normalize(x)
x_positive_test = normalize(x_positive_test)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
print("Num Train:\t\t", len(x_train))
print("Num Test:\t\t", len(x_test))
print(f"Percent Negative Train:\t {100 * sum(y_train == 0.0) / len(y_train):.1f}")
print(f"Percent Negative Test:\t {100 * sum(y_test == 0.0) / len(y_test):.1f}")

In [ ]:
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
x_positive_test = np.expand_dims(x_positive_test, -1)

num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_positive_test = keras.utils.to_categorical(y_positive_test, num_classes)

In [ ]:
input_shape = np.shape(x_train[0])
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv1D(32, kernel_size=(3), activation="relu"),
        #layers.MaxPooling2D(pool_size=(2)),
        layers.Conv1D(64, kernel_size=(3), activation="relu"),
        #layers.MaxPooling2D(pool_size=(2)),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy", 
              optimizer="adam", 
              metrics=["accuracy"],
              #loss_weights = sum(y_train) / len(y_train),
              #weighted_metrics = ['accuracy']
             )

In [ ]:
x_positive_test.shape

In [ ]:
np.concatenate((x_positive_test, x_test[:5000])).shape

In [ ]:
negative_weight, positive_weight = class_weight.compute_class_weight('balanced', 
                                                           classes = np.unique(y_train),
                                                           y = y_train[:,1])

print(f"Negative Weight: {negative_weight:.2f}")
print(f"Positive Weight: {positive_weight:.2f}")

In [ ]:
from sklearn.utils import class_weight

weight_offset = 200000
negative_weight = (1 - (sum(y_train) - weight_offset) / len(y_train))[0]
positive_weight = (1 - (sum(y_train) + weight_offset) / len(y_train))[1]

negative_weight, positive_weight = class_weight.compute_class_weight('balanced', 
                                                           classes = np.unique(y_train),
                                                           y = y_train[:,1])

print(f"Negative Weight: {negative_weight:.2f}")
print(f"Positive Weight: {positive_weight:.2f}")

batch_size = 128
epochs = 60

model.fit(x_train, 
          y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data = (np.concatenate((x_positive_test, x_test[:5000])), 
                             np.concatenate((y_positive_test, y_test[:5000]))),
          #validation_split=0.1,
          #class_weight = {0: negative_weight, 1: positive_weight}
         )

plt.figure(figsize=(8,5), dpi=100)
plt.plot(model.history.history['accuracy'], label='Train Acc')
plt.plot(model.history.history['val_accuracy'], c='r', label='Val Acc')
plt.xlabel('Epoch')
plt.xticks(range(epochs))
plt.ylabel('Accuracy')
plt.legend()
plt.title('Network Train and Val Accuracy')
plt.show()

In [ ]:
keras.models.save_model(model, '../models/model-12-04-2020.h5')

In [ ]:
convs = []
for filter in range(32):
    convs.append([dim[0][filter] for dim in model.layers[0].get_weights()[0]])

In [ ]:
out = []
out.append([[np.dot(x_test[0,i:3+i,0], np.array(conv)) for i in range(10)] for conv in convs])

In [ ]:
plt.figure(figsize=(20,3), facecolor=(1,1,1))
plt.imshow(np.array(convs).T, cmap='seismic', vmin=-1, vmax=1)
plt.title('Convolution Kernels')
plt.xticks(range(32))
plt.xlabel('Convolution Index')
plt.colorbar()

for sample in range(10):
    out = []
    out.append([[np.dot(x_test[sample,i:3+i,0], np.array(conv)) for i in range(10)] for conv in convs])
    plt.figure(figsize=(20,3), facecolor=(1,1,1))
    plt.imshow(np.concatenate((np.transpose(x_test[sample,1:11]), out[0][:])).T, cmap='seismic', vmin=-1, vmax=1)
    plt.colorbar()
    plt.title('First Layer Convolution Outputs - Sample ' + str(sample) + ', Class: ' + str(int(y_test[sample][1])))
    plt.xticks(range(33), ['input'] + [str(_) for _ in range(32)])
    plt.xlabel('Input and Convolved Outputs by Conv Index')
    plt.show()

In [ ]:
sample_1 = 3
sample_2 = 4

a_out = []
a_out.append([[np.dot(x_test[sample_1,i:3+i,0], np.array(conv)) for i in range(10)] for conv in convs])
b_out = []
b_out.append([[np.dot(x_test[sample_2,i:3+i,0], np.array(conv)) for i in range(10)] for conv in convs])
a = np.concatenate((np.transpose(x_test[sample_1,1:11]), a_out[0][:])).T
b = np.concatenate((np.transpose(x_test[sample_2,1:11]), b_out[0][:])).T
plt.figure(figsize=(20,3), facecolor=(1,1,1))
plt.imshow(a-b, cmap='seismic', vmin=-1, vmax=1)
plt.xticks(range(33), ['input'] + [str(_) for _ in range(32)])
plt.xlabel('Input and Conv Filter Indices')
plt.colorbar()
plt.title(f'Convolution Output Difference between Samples {sample_1} and {sample_2}')
plt.show()

In [ ]:
for i in range(10):
    plt.plot(x_test[i], label='Not TPA (sample 3)')
plt.show()

In [ ]:
plt.figure(figsize=(8,5), dpi=150, facecolor=(1,1,1))
plt.plot(x_test[0], label='Not TPA (sample 3)')
plt.plot(x_test[sample_2], color='r', label='TPA (sample 4)')
plt.legend()
plt.xticks(range(len(band_descriptions)), band_descriptions)
plt.xlabel('Band')
plt.ylabel('Normalized Reflectance')
plt.title('Spectral Profile of Two Pixel Vectors')
plt.show()

In [ ]:
for conv in convs:
    plt.plot(conv, alpha=0.3, color='k')
plt.show()

In [ ]:
convs[0]

In [ ]:
plt.figure(figsize=(20,3))
plt.imshow(np.array(convs).T, cmap='seismic', vmin=-1, vmax=1)
plt.colorbar()

In [ ]:
plt.figure(figsize=(20,3))
plt.imshow(np.array(convs).T, cmap='seismic', vmin=-1, vmax=1)
plt.colorbar()

In [ ]:
plt.plot(model.history.history['weighted_accuracy'], label='Train Acc')
plt.plot(model.history.history['val_weighted_accuracy'], c='r', label='Val Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
test_metrics = model.evaluate(x_positive_test, y_positive_test)
print(f"Test Loss: {test_metrics[0]:.2f}\nTest Accuracy: {test_metrics[1]:.2f}")

In [ ]:
[element[0] for element in y_test[:num_samples]]

In [ ]:
num_samples = 50
plt.figure(figsize=(20, 3), dpi=200, facecolor=(1,1,1))
plt.imshow(x_train[:num_samples,:,0].T, cmap='viridis')
plt.ylabel("Band")
plt.xlabel("Class Label")
plt.yticks(range(len(band_descriptions)), band_descriptions)
plt.xticks(range(num_samples), labels=[int(element[1]) for element in y_train[:num_samples]])
plt.colorbar()
plt.title("Input Training Data")
plt.show()

In [ ]:
num_samples = 16
plt.figure(figsize=(20, 3), dpi=200, facecolor=(1,1,1))
plt.imshow(x_positive_test[:num_samples,:,0].T, cmap='viridis')
plt.ylabel("Band")
plt.colorbar()
plt.yticks(range(len(band_descriptions)), band_descriptions)
plt.xticks(range(num_samples), labels=[int(element[1]) for element in y_positive_test[:num_samples]])
plt.show()

In [ ]:
model.save('../models/model-12-02-2020.h5')

In [ ]:
# Sentinel 2 band descriptions
band_descriptions = {
    'B1': 'Aerosols, 442nm',
    'B2': 'Blue, 492nm',
    'B3': 'Green, 559nm',
    'B4': 'Red, 665nm',
    'B5': 'Red Edge 1, 704nm',
    'B6': 'Red Edge 2, 739nm',
    'B7': 'Red Edge 3, 779nm',
    'B8': 'NIR, 833nm',
    'B8A': 'Red Edge 4, 864nm',
    'B9': 'Water Vapor, 943nm',
    'B11': 'SWIR 1, 1610nm',
    'B12': 'SWIR 2, 2186nm'
}

band_wavelengths = [442, 492, 559, 665, 704, 739, 779, 833, 864, 943, 1610, 2186]

In [ ]:
def get_pixel_vectors(data_source, month):
    pixel_vectors = []
    for site in data_source[list(data_source.keys())[0]]:
        #for month in data_source.keys():
        if -999 not in data_source[month][site]['B2']:
            width, height = np.shape(data_source[month][site]['B2'])
            for i in range(width):
                for j in range(height):
                    pixel_vector = []
                    for band in band_descriptions:
                        pixel_vector.append(data_source[month][site][band][i][j])
                    pixel_vectors.append(pixel_vector)
        else: width, height = 0, 0
    return pixel_vectors, width, height

In [ ]:
def make_predictions(site_name, threshold):
    with open(os.path.join(DATA_DIR, site_name + "_0.03_patch.pkl"), 'rb') as file:
        test_image = pickle.load(file)

    rgb_stack = []
    preds_stack = []
    threshold_stack = []

    for month in tqdm(list(test_image.keys())):
        test_pixel_vectors, width, height = get_pixel_vectors(test_image, month)
        if width > 0:
            test_pixel_vectors = normalize(test_pixel_vectors)

            r = np.reshape(np.array(test_pixel_vectors)[:,3], (width, height))
            g = np.reshape(np.array(test_pixel_vectors)[:,2], (width, height))
            b = np.reshape(np.array(test_pixel_vectors)[:,1], (width, height))
            rgb = np.moveaxis(np.stack((r,g,b)), 0, -1)
            rgb_stack.append(rgb)

            preds = model.predict(np.expand_dims(test_pixel_vectors, axis=-1))
            preds_img = np.reshape(preds, (width, height, 2))[:,:,1]
            preds_stack.append(preds_img)

            thresh_img = preds_img > threshold
            threshold_stack.append(thresh_img)
    
    output_dir = './figures/neural_network/12-04-2020'
    if not os.path.exists(output_dir):
            os.mkdir(output_dir)

            
    rgb_median = np.median(rgb_stack, axis=0)
    preds_median = np.median(preds_stack, axis=0)
    threshold_median = np.median(threshold_stack, axis=0)
    
    plt.figure(dpi=150, facecolor=(1,1,1), figsize=(15,5))

    plt.subplot(1,3,1)
    plt.imshow(rgb_median / np.max(rgb_median))
    plt.title(f'{site_name} Median', size=8)
    plt.axis('off')

    plt.subplot(1,3,2)
    plt.imshow(preds_median, vmin=0, vmax=1, cmap='seismic')
    plt.title('Classification Median', size=8)
    plt.axis('off')

    plt.subplot(1,3,3)
    plt.imshow(threshold_median, vmin=threshold, vmax=1, cmap='gray')
    plt.title(f"Positive Pixels Median: Threshold {threshold}", size=8)
    plt.axis('off')

    title = f"{site_name} Test Set - Median Values - Neural Network Classification - Threshold {threshold}"
    plt.suptitle(title, y=1.01)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, title + '.png'), bbox_inches='tight')
    plt.show()
    
    
    fig, ax = plt.subplots(dpi=200, facecolor=(1,1,1), figsize=(4,4))
    ax.set_axis_off()
    clipped_img = np.moveaxis([channel * (preds_median > 0) for channel in np.moveaxis(rgb_median, -1, 0)], 0, -1)
    img = plt.imshow(clipped_img / (clipped_img.max()))
    ax.set_title('Threshold 0')
    plt.tight_layout()

    def animate(i):
        i /= 100
        clipped_img = np.moveaxis([channel * (preds_median > i) for channel in np.moveaxis(rgb_median, -1, 0)], 0, -1)
        img.set_data(clipped_img / (clipped_img.max()))
        #img.set_data((preds_stack > i) * 1)
        ax.set_title(site_name + ' Threshold ' + str(i))
        return img,

    ani = animation.FuncAnimation(fig, animate, frames=100, interval=60, blit=True, repeat_delay=500)
    ani.save(os.path.join(output_dir, site_name + 'test_set_threshold_visualization' + '.mp4'))
    plt.close()
    
    return rgb_median, preds_median, threshold_median
    

In [ ]:
model = keras.models.load_model('../models/model-12-02-2020.h5')

In [ ]:
DATA_DIR = '../data'
site_names = ['bare_earth_4', 'city_7', 'tpa_babandem', 'tpa_bangli', 'tpa_biaung', 'tpa_mandung', 'tpa_jimbaran', 'tpa_jimbaran']
threshold = 0.90

for site_name in site_names:
    rgb_median, preds_median, threshold_median = make_predictions(site_name, threshold)

In [ ]:
DATA_DIR = '../data'
site_names = ['tpa_mandung']
threshold = 0.91

for site_name in site_names:
    rgb_median, preds_median, threshold_median = make_predictions(site_name, threshold)

In [ ]:

plt.hist(np.array(preds_median).flatten(), 150, log=True);

In [ ]:
old_model = keras.models.load_model('../models/model-12-02-2020.h5')

In [ ]:
inputs = np.concatenate((x_positive_test, x_test[:20000]))
preds = model.predict(inputs)
labels = np.concatenate((y_positive_test, y_test[:20000]))

In [ ]:
sum(np.array(labels)[:,0] == 1.0) / len(labels)

In [ ]:
offset = 3300
num_samples = 500
threshold = 0.9
plt.figure(figsize=(8,5), dpi=100)
for pred, label, spectra in zip(preds[offset:offset+num_samples], labels[offset:offset+num_samples], inputs[offset:offset+num_samples]):
    # If label is negative
    if label[0] == 1.0:
        # If negative prediction is greater than threshold
        # plot lines without points
        if pred[0] >= threshold:
            plt.plot(spectra, color='green', alpha=0.05)
        # If negative prediction is below the threshold
        # plot line in yellow
        else:
            plt.plot(spectra, color='orange', alpha=0.25)
    # If label is positive
    else:
        # if prediction 
        if pred[1] >= threshold:
            plt.plot(spectra, color='C0', alpha=0.05)
        else:
            plt.plot(spectra, color='red', alpha=.25)
plt.show()

In [ ]:
fp_pred = []
fp_input = []
fp_rgb = []
threshold = 0.9
for pred, label, spectra in zip(preds, labels, inputs):
    if label[1] == 0 and pred[1] > threshold:
        fp_pred.append(pred)
        fp_input.append(spectra)
        rgb = np.zeros((1,1,3))
        rgb[0,0,:] = [spectra[3], spectra[2], spectra[1]]
        fp_rgb.append([spectra[3], spectra[2], spectra[1]])

fp_rgb = (np.squeeze(fp_rgb) - np.min(fp_rgb)) / (np.max(fp_rgb) - np.min(fp_rgb))
print(f"{100 * len(fp_pred) / sum(np.array(labels)[:,1] == 0):.2f}% of negative samples in the test dataset are classified as positives")

In [ ]:
tn_pred = []
tn_input = []
tn_rgb = []
threshold = 0.9
for pred, label, spectra in zip(preds, labels, inputs):
    if label[1] == 0 and pred[1] < threshold:
        tn_pred.append(pred)
        tn_input.append(spectra)
        rgb = np.zeros((1,1,3))
        rgb[0,0,:] = [spectra[3], spectra[2], spectra[1]]
        tn_rgb.append([spectra[3], spectra[2], spectra[1]])

tn_rgb = (np.squeeze(tn_rgb) - np.min(tn_rgb)) / (np.max(tn_rgb) - np.min(tn_rgb))
print(f"{100 * len(tn_pred) / sum(np.array(labels)[:,1] == 0):.2f}% of negative samples in the test dataset are correctly classified as negatives")

In [ ]:
fn_pred = []
fn_input = []
fn_rgb = []
threshold = 0.9
for pred, label, spectra in zip(preds, labels, inputs):
    if label[1] == 1 and pred[1] < threshold:
        fn_pred.append(pred)
        fn_input.append(spectra)
        rgb = np.zeros((1,1,3))
        rgb[0,0,:] = [spectra[3], spectra[2], spectra[1]]
        fn_rgb.append([spectra[3], spectra[2], spectra[1]])

fn_rgb = (np.squeeze(fn_rgb) - np.min(fn_rgb)) / (np.max(fn_rgb) - np.min(fn_rgb))
print(f"{100 * len(fn_pred) / sum(np.array(labels)[:,1] == 1):.0f}% of positive samples in the test dataset are incorrectly classified as negatives")

In [ ]:
tp_pred = []
tp_input = []
tp_rgb = []
threshold = 0.9
for pred, label, spectra in zip(preds, labels, inputs):
    if label[1] == 1 and pred[1] > threshold:
        tp_pred.append(pred)
        tp_input.append(spectra)
        rgb = np.zeros((1,1,3))
        rgb[0,0,:] = [spectra[3], spectra[2], spectra[1]]
        tp_rgb.append([spectra[3], spectra[2], spectra[1]])

tp_rgb = (np.squeeze(tp_rgb) - np.min(tp_rgb)) / (np.max(tp_rgb) - np.min(tp_rgb))
print(f"{100 * len(tp_pred) / sum(np.array(labels)[:,1] == 1):.0f}% of positive samples in the test dataset are correctly classified as positives")

In [ ]:
np.min(tn_rgb)

In [ ]:
plt.figure(figsize=(50,3), dpi=350, facecolor=(1,1,1))
plt.scatter(range(len(tn_pred)), sorted(np.array(tn_pred)[:,1]), c=np.squeeze(tn_rgb), s=1)
plt.xticks([])
plt.ylabel("Prediction Score")
plt.show()

In [ ]:
for i in range(12):
    plt.figure(figsize=(8,5), dpi=150)
    plt.scatter(np.array(fn_pred)[:,1], np.array(fn_input)[:,i],
                c=np.squeeze(fn_rgb),
                s=5
               )
    plt.ylabel(f'Band {i + 1} Reflectance (Green)')
    plt.xlabel('Classification Score')
    plt.title('False Positives')
    plt.show()

In [ ]:
def plot_pixel_grid(preds, colors, title):
    plt.figure(figsize=(20,20), dpi=50)
    num_samples = np.min([len(preds), 100])
    indices = np.random.choice(len(preds), num_samples)
    for i in range(num_samples):
        index = indices[i]
        pred = preds[index]
        rgb = np.expand_dims(np.expand_dims(colors[index], axis=0), axis=0)
        plt.subplot(10,10,i + 1)
        plt.imshow(rgb)
        plt.axis('off')
        plt.title(f"{pred[1]:.2f}")
    plt.suptitle(title, size=20, y=1.01)
    plt.tight_layout()
    plt.show()

In [ ]:
np.random.choice(2, 3)

In [ ]:
plot_pixel_grid(tp_pred, tp_rgb, 'Colors of True Positives')

In [ ]:
plot_pixel_grid(fp_pred, fp_rgb, 'Colors of False Positives')

In [ ]:
plot_pixel_grid(tn_pred, tn_rgb, 'Colors of True Negatives')

In [ ]:
plot_pixel_grid(fn_pred, fn_rgb, 'Colors of False Negatives')

In [ ]:
plt.figure(figsize=(20,20), dpi=50)
for i in range(100):
    index = i
    plt.subplot(10,10,i + 1)
    plt.imshow(np.expand_dims(np.expand_dims(np.stack((inputs[index][3][0], inputs[index][2][0], inputs[index][1][0])), axis=0), axis=0))
    plt.axis('off')
    plt.title(f"{preds[index][1]:.2f}")
plt.tight_layout()
plt.show()

In [ ]:
np.expand_dims(np.expand_dims(np.stack((inputs[index][3][0], inputs[index][2][0], inputs[index][1][0])), axis=0), axis=0).shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import tree


clf = tree.DecisionTreeClassifier(max_leaf_nodes=512, max_depth=64)
                                  #, class_weight='balanced')
clf = clf.fit(np.squeeze(x_train), y_train[:,1])

print("Accuracy:", clf.score(np.squeeze(x_test), y_test[:,1]))
print("Feature Importances:")
for band, importance in zip(band_descriptions, clf.feature_importances_):
    print(f"{band}: {importance:.3f}")

In [ ]:
site_name = 'tpa_mandung'
with open(os.path.join(DATA_DIR, site_name + "_0.03_patch.pkl"), 'rb') as file:
    test_image = pickle.load(file)

rgb_stack = []
preds_stack = []
threshold_stack = []

for month in tqdm(list(test_image.keys())):
    test_pixel_vectors, width, height = get_pixel_vectors(test_image, month)
    if width > 0:
        test_pixel_vectors = normalize(test_pixel_vectors)

        r = np.reshape(np.array(test_pixel_vectors)[:,3], (width, height))
        g = np.reshape(np.array(test_pixel_vectors)[:,2], (width, height))
        b = np.reshape(np.array(test_pixel_vectors)[:,1], (width, height))
        rgb = np.moveaxis(np.stack((r,g,b)), 0, -1)
        rgb_stack.append(rgb)

        preds = clf.predict(test_pixel_vectors)
        preds_img = np.reshape(preds, (width, height))
        preds_stack.append(preds_img)

        thresh_img = preds_img > threshold
        threshold_stack.append(thresh_img)

In [ ]:
rgb_median = np.median(rgb_stack, axis=0)
preds_median = np.median(preds_stack, axis=0)
threshold_median = np.median(threshold_stack, axis=0)

plt.figure(dpi=150, facecolor=(1,1,1), figsize=(15,5))

plt.subplot(1,3,1)
plt.imshow(rgb_median / np.max(rgb_median))
plt.title(f'{site_name} Median', size=8)
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(preds_median, vmin=0, vmax=1, cmap='seismic')
plt.title('Classification Median', size=8)
plt.axis('off')

plt.subplot(1,3,3)
plt.imshow(threshold_median, vmin=0, vmax=1, cmap='gray')
plt.title(f"Positive Pixels Median: Threshold {threshold}", size=8)
plt.axis('off')

title = f"{site_name} - Median Values - Decision Tree Classification - Threshold {threshold}"
plt.suptitle(title, y=1.01)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, title + '.png'), bbox_inches='tight')
plt.show()

fig, ax = plt.subplots(dpi=200, facecolor=(1,1,1), figsize=(4,4))
ax.set_axis_off()
clipped_img = np.moveaxis([channel * (preds_median > 0) for channel in np.moveaxis(rgb_median, -1, 0)], 0, -1)
img = plt.imshow(clipped_img / (clipped_img.max()))
ax.set_title('Threshold 0')
plt.tight_layout()

def animate(i):
    i /= 100
    clipped_img = np.moveaxis([channel * (preds_median > i) for channel in np.moveaxis(rgb_median, -1, 0)], 0, -1)
    img.set_data(clipped_img / (clipped_img.max()))
    #img.set_data((preds_stack > i) * 1)
    ax.set_title(site_name + ' Threshold ' + str(i))
    return img,

ani = animation.FuncAnimation(fig, animate, frames=101, interval=60, blit=True, repeat_delay=500)
ani.save(os.path.join(output_dir, site_name + '_decision_tree_threshold_visualization' + '.mp4'))
plt.close()

In [ ]:
plt.hist(np.array(x_train).flatten(), 100, log=True);

In [ ]:
output_dir = './figures/neural_network'
if not os.path.exists(output_dir):
        os.mkdir(output_dir)

threshold = 0.75

rgb_stack = []
classification_stack = []
threshold_stack = []

for month in list(test_image.keys()):
    test_pixel_vectors = get_pixel_vectors(test_image, month)
    if len(test_pixel_vectors) > 0:
        test_pixel_vectors = normalize(test_pixel_vectors)

        r = np.reshape(np.array(test_pixel_vectors)[:,3], (335, 335))
        g = np.reshape(np.array(test_pixel_vectors)[:,2], (335, 335))
        b = np.reshape(np.array(test_pixel_vectors)[:,1], (335, 335))
        rgb = np.moveaxis(np.stack((r,g,b)), 0, -1)
        
        plt.figure(dpi=150, facecolor=(1,1,1), figsize=(15,5))
        
        plt.subplot(1,3,1)
        plt.imshow(rgb ** 0.65)
        plt.title(month, size=8)
        plt.axis('off')
        
        plt.subplot(1,3,2)
        preds = model.predict(np.expand_dims(test_pixel_vectors, axis=-1))
        preds_img = np.reshape(preds, (335, 335, 2))[:,:,1]
        im = plt.imshow(preds_img, vmin=0, vmax=1, cmap='seismic')
        plt.axis('off')
        plt.title('Classification', size=8)
        #plt.colorbar(im,fraction=0.045, pad=0.02, ticks=[0, 1])
        
        plt.subplot(1,3,3)
        plt.imshow(preds_img > threshold, vmin=0, vmax=1, cmap='gray')
        plt.axis('off')
        plt.title(f"Positive Pixels: Threshold {threshold}", size=8)
        
        title = f"TPA Regional Bangli - Neural Network Classification - 0.03° Patch"
        plt.suptitle(title, y=0.9)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, title + '_' + month + '.png'), bbox_inches='tight')
        plt.show()

In [ ]:
clipped_img = np.moveaxis([channel * (preds_median > 0.98) for channel in np.moveaxis(rgb_median, -1, 0)], 0, -1)
plt.figure(dpi=300)
plt.imshow((clipped_img / (clipped_img.max())))
plt.axis('off')